Imports

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

1. набор данных не сбалансированный по целевому признаку и если обучать модели на этом наборе получится хорошее качество для класса 0 и  плохое качество для класса 1. Поэтому надо применять методы для балансировки данных: уменьшите размер мажоритарного (преобладающий) класса  и при необходимости сгенерируйте данные для миноритарного (маленького) класса, например, с помощью метода smote

In [ ]:
# Загрузка данных
data = pd.read_csv('your_file.csv')  # Укажите путь к вашему CSV файлу

# Разделение данных на признаки и целевую переменную
X = data.drop(columns=['target'])  # 'target' - имя целевой переменной
y = data['target']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Проверка баланса классов до балансировки
print(f'Распределение классов до балансировки: {Counter(y_train)}')

# 1. Балансировка с помощью уменьшения размера мажоритарного класса (RandomUnderSampler)
under_sampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
X_train_under, y_train_under = under_sampler.fit_resample(X_train, y_train)

# Проверка баланса классов после уменьшения мажоритарного класса
print(f'Распределение классов после уменьшения мажоритарного класса: {Counter(y_train_under)}')

# 2. Балансировка с помощью генерации данных для миноритарного класса (SMOTE)
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Проверка баланса классов после применения SMOTE
print(f'Распределение классов после применения SMOTE: {Counter(y_train_smote)}')

model = RandomForestClassifier(random_state=42)
model.fit(X_train_smote, y_train_smote)

# Оценка качества модели на тестовой выборке
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

2. При анализе качества используйте не только метрики, но и матрицу погрешностей и отчет по метрикам качества классификации. Например, отчет по качеству

Accuracy: 94.58375125376128
Classification Report:               
               precision    recall  f1-score   support

           0       0.95      1.00      0.97       943
           1       0.00      0.00      0.00        54

    accuracy                           0.95       997
   macro avg       0.47      0.50      0.49       997
weighted avg       0.89      0.95      0.92       997
говорит, что обучение проведено крайне плохо, т.к. f1-score по классу 1 равно 0

In [ ]:
# Оценка качества модели на тестовой выборке
y_pred = model.predict(X_test)

# Матрица ошибок
conf_matrix = confusion_matrix(y_test, y_pred)
print("Матрица ошибок:")
print(conf_matrix)

# Отчет по метрикам классификации
class_report = classification_report(y_test, y_pred)
print("Отчет по метрикам качества:")
print(class_report)

# Точность модели (accuracy)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

3. У вас должно быть не менее 3-х различных методов МО для решения задачи. Примеры методов: KNN, деревья решений, случайный лес, логистическая регрессия, метод опорных векторов, градиентный бустинг

In [ ]:
# Список моделей для обучения
models = {
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "SVM": SVC(random_state=42)
}

# Функция для обучения и оценки модели
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Обучение модели
    model.fit(X_train, y_train)
    
    # Предсказания на тестовых данных
    y_pred = model.predict(X_test)
    
    # Оценка качества
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    class_report = classification_report(y_test, y_pred)
    
    # Вывод результатов
    print(f"Результаты для модели {model.__class__.__name__}:")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print("Матрица ошибок:")
    print(conf_matrix)
    print("Отчет по метрикам качества:")
    print(class_report)
    print("-" * 50)

# Обучение и оценка каждой модели
for name, model in models.items():
    train_and_evaluate_model(model, X_train, y_train, X_test, y_test)